In [1]:
%matplotlib inline

import os
import glob
import io

from math import atan2, degrees

import imageio

from PIL import Image

import numpy as np
import pandas as pd

import matplotlib as mpl
from matplotlib import pyplot as plt

from scipy.ndimage import gaussian_filter

In [2]:
def get_data(mediafile):
    media = imageio.get_reader("../Dataset/Mahnob/data/media_24/{}.avi".format(mediafile))
    valence_fixations = pd.DataFrame.from_csv("data/valence_data_for_heatmap_creation_{}.csv".format(mediafile)).applymap(eval)
    arousal_fixations = pd.DataFrame.from_csv("data/arousal_data_for_heatmap_creation_{}.csv".format(mediafile)).applymap(eval)
    return media, valence_fixations, arousal_fixations

In [3]:
# matplotlib helper function
plt.ioff()

dpi = 100
DISPLAY_SIZE = (320,200)
figsize = (DISPLAY_SIZE[0]/dpi, DISPLAY_SIZE[1]/dpi)
extent = [0, DISPLAY_SIZE[0], DISPLAY_SIZE[1], 0]

def create_ax():
    # determine the figure size in inches
    fig = plt.figure(figsize=figsize, frameon=False)
    ax = fig.gca()
    ax.axis(extent)
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    return fig, ax

def export(fix, ax):
    self.main_axis.imshow(self.base, **self.base_opts)
    for overlay, opts in zip(self.overlays, self.overlays_opts):
        self.main_axis.imshow(overlay, **opts)
    self.main_axis.figure.canvas.draw()
    width, height = self.main_figure.get_size_inches() * self.main_figure.get_dpi()
    image = np.fromstring(self.main_axis.figure.canvas.tostring_rgb(), dtype='uint8').reshape(int(height), int(width), 3)
    return image

def fig2data(fig):
    """
    @brief Convert a Matplotlib figure to a 4D numpy array with RGBA channels and return it
    @param fig a matplotlib figure
    @return a numpy 3D array of RGBA values
    """
    # draw the renderer
    fig.canvas.draw()
    # Get the RGBA buffer from the figure
    w, h = fig.get_size_inches() * fig.get_dpi()
    buf = np.fromstring(fig.canvas.tostring_argb(), dtype=np.uint8)
    buf.shape = (h, w, 4)
    # canvas.tostring_argb give pixmap in ARGB mode. Roll the ALPHA channel to have it in RGBA mode
    buf = np.roll(buf, 3, axis=2)
    return buf

def fig2ImageData(fig, ax):
    fig.canvas.draw()
    with io.BytesIO() as imgdata:
        fig.savefig(imgdata, format='png', dpi=fig.dpi, bbox_inches='tight', transparent=True)
        imgdata.seek(0)
        im = Image.open(imgdata)
        im = np.array(im.getdata())
    return im

In [4]:
# utility function
 
def visual_span_from_distance(distance):
    h = 324 # Monitor height in cm
    d = distance # Distance between monitor and participant in cm
    r = 800 # Vertical resolution of the monitor
    size_in_deg = 5. # The stimulus size in pixels
    # Calculate the number of degrees that correspond to a single pixel. This will
    # generally be a very small value, something like 0.03.
    deg_per_px = degrees(atan2(.5*h, d)) / (.5*r)
    # Calculate the size of the stimulus in degrees
    size_in_px = size_in_deg / deg_per_px
    return size_in_px

def normalize(data):
    return ( data - data.min() ) / ( data.max() - data.min() )

In [5]:
def create_image_from_fixation(x, y, v, distance=None):
    m = np.zeros([DISPLAY_SIZE[1], DISPLAY_SIZE[0]], dtype=float)
    m[int(y), int(x)] = v
    
    if distance is None:
        sigma = 12
    else:
        sigma = 0.17 * visual_span_from_distance(distance) / 4
    
    m = gaussian_filter(m, sigma=sigma)
    return m

def compute_frame_saliency_map(data):
    ## matrix of zeros
    M = np.zeros([DISPLAY_SIZE[1], DISPLAY_SIZE[0]], dtype=float)
    # gaussian matrix
    for x, y, duration, distance, _ in data:
        if 0<=x<=DISPLAY_SIZE[0] and 0<=y<=DISPLAY_SIZE[1]:
            m = create_image_from_fixation(x, y, v=duration, distance=distance)
            M = np.add(M, m)
    return M

def compute_frame_emotion_saliency_map(data):
    ## matrix of zeros
    M = np.zeros([DISPLAY_SIZE[1], DISPLAY_SIZE[0]], dtype=float)
    # gaussian matrix
    for x, y, duration, distance, feature in data:
        if 0<=x<=DISPLAY_SIZE[0] and 0<=y<=DISPLAY_SIZE[1]:
            m = create_image_from_fixation(x, y, v=feature, distance=distance)
            M = np.add(M, m)
    return M

def get_frame_saliency_maps(gaze_data, frames, compute_fnc):
    frame_saliency_maps = []
    
    for frame in frames:
                
        gd = gaze_data.T[frame]
        frame_saliency_maps.append(compute_fnc(gd))
        
    frame_saliency_maps = np.array(frame_saliency_maps)
    
    # apply normalization to get the information of the relation with other heatmaps
    frame_saliency_maps = normalize(frame_saliency_maps)
    
    # apply some threashold to remove useless peak
    #frame_saliency_maps[frame_saliency_maps<=.2] = 0
    
    return frame_saliency_maps


def compute_fixation_scatter(fix):
    fig, ax = create_ax()
    for f in fix:
        x, y, d, distance, _ = f
        ax.scatter(x, y, s=2, alpha=.8, c='orange')
        r = visual_span_from_distance(distance) / 4
        circle = plt.Circle((x, y), 1e-2*d, alpha=.8, color='orange', fill=False)
        ax.add_artist(circle)
    return fig2ImageData(fig, ax)

def get_fixations_scatter(gaze_data, frames):

    fixations_scatter = []
    for frame in frames:
        fix = gaze_data.T[frame]
        fixations_scatter.append(compute_fixation_scatter(fix=fix))

    fixations_scatter = np.array(fixations_scatter)
    
    return fixations_scatter


In [6]:
def export_gaze_heatmap(mediafile):

    print("Exporting data for mediafile {}".format(mediafile))
    
    media, valence_fixations, arousal_fixations = get_data(mediafile)
    number_of_frame = media.get_meta_data()['nframes']
    
    #fixations_scatter  = get_fixations_scatter(fixations, range(number_of_frame))
    
    fixation_heatmaps = get_frame_saliency_maps(valence_fixations, range(number_of_frame), compute_frame_saliency_map)
    np.savez_compressed("data/valence_fixation_heatmap_{}.npz".format(mediafile), heatmaps=fixation_heatmaps)
    del fixation_heatmaps
    print("... valence_fixation_heatmap done.")
    
    emotion_heatmaps = get_frame_saliency_maps(valence_fixations, range(number_of_frame), compute_frame_emotion_saliency_map)
    np.savez_compressed("data/valence_emotion_heatmap_{}.npz".format(mediafile), heatmaps=emotion_heatmaps)
    del emotion_heatmaps
    print("... valence_emotion_heatmap done.")
    print()
    
    fixation_heatmaps = get_frame_saliency_maps(arousal_fixations, range(number_of_frame), compute_frame_saliency_map)
    np.savez_compressed("data/arousal_fixation_heatmap_{}.npz".format(mediafile), heatmaps=fixation_heatmaps)
    del fixation_heatmaps
    print("... arousal_fixation_heatmap done.")
    
    emotion_heatmaps = get_frame_saliency_maps(arousal_fixations, range(number_of_frame), compute_frame_emotion_saliency_map)
    np.savez_compressed("data/arousal_emotion_heatmap_{}.npz".format(mediafile), heatmaps=emotion_heatmaps)
    del emotion_heatmaps
    print("... arousal_emotion_heatmap done.")
    print()
    
    del media
    del valence_fixations
    del arousal_fixations
    
    #data = {
        #"fixations_scatter"  : fixations_scatter,
    #    "fixations_heatmaps" : fixations_heatmaps,
    #    "emotion_heatmaps"   : emotion_heatmaps
    #}
    
    #return data-

In [7]:
for mediafile in [30, 53, 69, 90, 111]:
    data = export_gaze_heatmap(mediafile=mediafile)

Exporting data for mediafile 30


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  after removing the cwd from sys.path.


... valence_fixation_heatmap done.
... valence_emotion_heatmap done.

... arousal_fixation_heatmap done.
... arousal_emotion_heatmap done.

Exporting data for mediafile 53


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  after removing the cwd from sys.path.


... valence_fixation_heatmap done.
... valence_emotion_heatmap done.

... arousal_fixation_heatmap done.
... arousal_emotion_heatmap done.

Exporting data for mediafile 69


...

... valence_fixation_heatmap done.
... valence_emotion_heatmap done.

... arousal_fixation_heatmap done.
... arousal_emotion_heatmap done.

Exporting data for mediafile 90


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  after removing the cwd from sys.path.


... valence_fixation_heatmap done.
... valence_emotion_heatmap done.

... arousal_fixation_heatmap done.
... arousal_emotion_heatmap done.

Exporting data for mediafile 111


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  after removing the cwd from sys.path.


... valence_fixation_heatmap done.
... valence_emotion_heatmap done.

... arousal_fixation_heatmap done.
... arousal_emotion_heatmap done.

